In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import pandas as pd
import time
import lxml
#import cchardet
import concurrent.futures

In [44]:
website = "https://www.arabam.com/ikinci-el/otomobil?currency=TL&maxPrice=110000&minPrice=100000&take=50&view=Detailed"


In [45]:



headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"}

response = requests.get(website, headers=headers)

In [46]:
response.status_code

200

In [47]:
soup = BeautifulSoup(response.content,'html.parser')

In [8]:
len(results)

NameError: name 'results' is not defined

In [48]:
#results = soup.find_all("div", {"class":"content-container"})
results = soup.find_all("span", {"class":"bold"})

In [49]:
len(results)

4

# Id

In [11]:
results[50]

<div class="content-container">
<a href="{{advertUrl}}?utm_source=recommendation&amp;utm_medium=click&amp;utm_campaign=listeleme" onclick="window.dataLayer.push({Category:'Recommendation', Action:'Listing', Label:'Click_listing', event: 'gaEvent'})" target="_self">
<div class="df df-jcsbetween">
<p class="bold">{{advertCity}}</p>
<p class="model-year semi-bold">{{advertYear}}</p>
</div>
<div class="model-name">{{advertTitle}}</div>
<div class="price-and-icon-wrapper tar">
<div class="price">{{advertFormmatedPrice}}</div>
</div>
</a>
</div>

In [50]:
results = soup.find_all("span", {"class":"bold"})
results[1].get_text()

'24'

In [12]:
for i in range(50):
    print(results[i].find('p',{"class":"id"}).get_text().strip())

İlan No: 21132580
İlan No: 21019561
İlan No: 21132048
İlan No: 21093071
İlan No: 21131104
İlan No: 21131195
İlan No: 21131146
İlan No: 20304267
İlan No: 21116671
İlan No: 21130508
İlan No: 21130515
İlan No: 21129994
İlan No: 21128110
İlan No: 21130266
İlan No: 20886966
İlan No: 21130169
İlan No: 21130146
İlan No: 21130078
İlan No: 21061304
İlan No: 21130032
İlan No: 21129929
İlan No: 21093853
İlan No: 21062374
İlan No: 20647512
İlan No: 21129831
İlan No: 21129780
İlan No: 21089577
İlan No: 21129608
İlan No: 21096551
İlan No: 21129712
İlan No: 21105690
İlan No: 21112498
İlan No: 21112445
İlan No: 21112435
İlan No: 21129271
İlan No: 21129210
İlan No: 21129214
İlan No: 21129205
İlan No: 21129192
İlan No: 21129167
İlan No: 21126635
İlan No: 21110362
İlan No: 21118350
İlan No: 21107061
İlan No: 21119559
İlan No: 21118399
İlan No: 21128334
İlan No: 20709930
İlan No: 20679689
İlan No: 20584087


In [13]:
id_list = list()
for elem in results[:-1:]:
    id = elem.find('p',{"class":"id"}).get_text().strip().split()[2]
    id_list.append(id)
    

# cars at pages

In [91]:
PAGE_NUMBER = 50
id_list = list()

for i in range(PAGE_NUMBER):
    webiste = f"https://www.arabam.com/ikinci-el/otomobil?take=50&view=Detailed&page={i+1}"
    response = requests.get(website, headers=headers)
    soup = BeautifulSoup(response.content,'html.parser')
    '''results = soup.find_all("div", {"class":"content-container"})
    for elem in results[:-1:]:
        id = elem.find('p',{"class":"id"}).get_text().strip().split()[2]
        id_list.append(id)
    '''
    
    
    

In [88]:
len(id_list)

2500

# car specific features

In [167]:
website = "https://www.arabam.com/ilan/20310549"
response = requests.get(website, headers=headers)
soup = BeautifulSoup(response.content,'html.parser')

In [172]:
results = soup.find_all("li",{"class":"bcd-list-item"})


AttributeError: ResultSet object has no attribute 'isempty'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [166]:
results[-1].get_text().strip().split()
car_dict = dict()
for elem in results:  
    features_car = ''.join(elem.get_text().split()).split(":")
    car_dict.update({features_car[0]:features_car[1]})
pd.DataFrame([car_dict])
    
    
        
    
    #print(elem.get_text().replace(" ","").strip())

,İlanNo,İlanTarihi,Marka,Seri,Model,Yıl,Kilometre,VitesTipi,YakıtTipi,KasaTipi,MotorHacmi,MotorGücü,Çekiş,Ort.YakıtTüketimi,YakıtDeposu,Boya-değişen,TakasaUygun,Kimden
0,20310569,11Ağustos2022,Skoda,Octavia,1.4TSISport,2018,115.000km,YarıOtomatik,Benzin,Hatchback/5,1395cc,150hp,ÖndenÇekiş,5lt,50lt,Tamamıorjinal,TakasaUygunDeğil,Sahibinden


In [31]:


start_time = time.time()
session_object = requests.Session()
headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"}
all_car_df = pd.DataFrame()
all_id = id_list
for car_id in all_id:
    website = f"https://www.arabam.com/ilan/{car_id}"
    response = session_object.get(website, headers=headers)
    soup = BeautifulSoup(response.text,'lxml')
    results = soup.find_all("li",{"class":"bcd-list-item"})
    if len(results) ==0:
        continue
    results[-1].get_text().strip().split()
    car_dict = dict()
    for elem in results:  
        features_car = ''.join(elem.get_text().split()).split(":")
        car_dict.update({features_car[0]:features_car[1]})
    car_df = pd.DataFrame([car_dict])
    all_car_df = pd.concat([car_df,all_car_df])

print("--- %s seconds ---" % (time.time() - start_time)) 

    

--- 20.138076066970825 seconds ---


In [26]:
all_car_df.reset_index(inplace=True)

In [29]:
MAX_THREADS = 500



22.588613748550415 seconds to download stories.


In [32]:
start_time = time.time()
for i in range(300000):
    x=i
end_time = time.time()
print(start_time - end_time)

-0.02048206329345703


In [55]:
# for loop filtering with mileage
# for loop iterating at specific mileage range
# getting ilan no 
# iterating ilan no and getting features of cars
def get_id(price_diff,min_price, max_price):
    cr_id_list = []
    start_time=time.time()
    for i in range(((max_price-min_price)//price_diff)):
        upper_lim = min_price + (price_diff)*(i+1)
        lower_lim = min_price + (price_diff)*i
        print(f"upper: {upper_lim}" + f" lower: {lower_lim}")
        session_object = requests.Session()
        headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"}
        website = f"https://www.arabam.com/ikinci-el/otomobil?currency=TL&maxPrice={upper_lim}&minPrice={lower_lim}&take=50&view=Detailed"
        response = session_object.get(website,headers=headers)
        soup = BeautifulSoup(response.text,'lxml')
        results = soup.find_all("span", {"class":"bold"})
        page_num = int(results[1].get_text())
        print(page_num)
        for i in range(page_num):
            website = f"https://www.arabam.com/ikinci-el/otomobil?currency=TL&maxPrice={upper_lim}&minPrice={lower_lim}&take=50&view=Detailed&page={i+1}"
            response = session_object.get(website,headers=headers)
            soup = BeautifulSoup(response.text,'lxml')
            results = soup.find_all("div", {"class":"content-container"})
            for elem in results[:-1:]:
                id = elem.find('p',{"class":"id"}).get_text().strip().split()[2]
                cr_id_list.append(id)
    end_time = time.time()
    print(print(end_time-start_time))
    return(cr_id_list)
            
            
                
            
        
        




all_id = get_id(10000,100000,200000)
    
    
    
    
    



upper: 110000 lower: 100000
24
upper: 120000 lower: 110000
34
upper: 130000 lower: 120000
40
upper: 140000 lower: 130000
40
upper: 150000 lower: 140000
43
upper: 160000 lower: 150000
45
upper: 170000 lower: 160000
50
upper: 180000 lower: 170000
50
upper: 190000 lower: 180000
50
upper: 200000 lower: 190000
50
106.43761086463928
None


In [57]:
len(all_id)

21156

In [58]:
set_of_id = set(all_id)

In [59]:
all_id =list(set_of_id)

17447

In [66]:
def get_car_features(all_id):
    start_time = time.time()
    session_object = requests.Session()
    headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"}
    all_car_df = pd.DataFrame()
    for x,car_id in enumerate(all_id):
        website = f"https://www.arabam.com/ilan/{car_id}"
        response = session_object.get(website, headers=headers)
        soup = BeautifulSoup(response.text,'lxml')
        results = soup.find_all("li",{"class":"bcd-list-item"})
        if len(results) ==0:
            continue
        results[-1].get_text().strip().split()
        car_dict = dict()
        for elem in results:  
            features_car = ''.join(elem.get_text().split()).split(":")
            car_dict.update({features_car[0]:features_car[1]})
        car_df = pd.DataFrame([car_dict])
        all_car_df = pd.concat([car_df,all_car_df])
        if x%100 ==0:
            print(str(x)+"th iteration")
        if x ==300:
            break
            
    

    print("--- %s seconds ---" % (time.time() - start_time)) 
    return(all_car_df)

In [67]:
all_car_df = get_car_features(all_id)

0th iteration
100th iteration
200th iteration
300th iteration
--- 119.97726583480835 seconds ---


In [68]:
len(all_car_df)

301

In [70]:
all_car_df.reset_index()

,index,İlanNo,İlanTarihi,Marka,Seri,Model,Yıl,Kilometre,VitesTipi,YakıtTipi,KasaTipi,MotorHacmi,MotorGücü,Çekiş,YakıtDeposu,Boya-değişen,TakasaUygun,Kimden,Ort.YakıtTüketimi
0,0,21074793,29Eylül2022,Renault,R19,1.6EuropaRNE,1998,247.000km,Düz,LPG&Benzin,Sedan,1598cc,90hp,ÖndenÇekiş,55lt,Belirtilmemiş,TakasaUygunDeğil,Sahibinden,NaN
1,0,21075160,29Eylül2022,Fiat,Palio,1.2S,2001,203.000km,Düz,LPG&Benzin,Hatchback/5,1242cc,74hp,ÖndenÇekiş,51lt,"3değişen,9boyalı",TakasaUygun,Sahibinden,"7,2lt"
2,0,21075144,29Eylül2022,Mercedes-Benz,E,200,1987,358.000km,Düz,LPG&Benzin,Sedan,1997cc,109hp,Arkadanİtiş,NaN,Belirtilmemiş,TakasaUygunDeğil,Sahibinden,NaN
3,0,21075036,29Eylül2022,Tofaş,Doğan,S,1994,85.000km,Düz,LPG&Benzin,Sedan,1581cc,83hp,Arkadanİtiş,50lt,Belirtilmemiş,TakasaUygunDeğil,Sahibinden,"6,5lt"
4,0,21075209,29Eylül2022,Renault,Megane,2.0RXT,1998,358.125km,Düz,LPG&Benzin,Sedan,1998cc,115hp,ÖndenÇekiş,60lt,Belirtilmemiş,TakasaUygunDeğil,Sahibinden,"8,4lt"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,0,20566589,17Eylül2022,Tofaş,Doğan,SLXie,2000,99.999km,Düz,LPG&Benzin,Sedan,1581cc,96hp,Arkadanİtiş,45lt,Belirtilmemiş,TakasaUygun,Galeriden,"6,6lt"
297,0,21005745,19Eylül2022,Opel,Vectra,2.0CD,1996,228.500km,Düz,LPG&Benzin,Sedan,1998cc,115hp,ÖndenÇekiş,NaN,"2değişen,11boyalı",TakasaUygunDeğil,Sahibinden,NaN
298,0,21106260,04Ekim2022,Daewoo,Nubira,1.6SX,1998,275.000km,Düz,LPG&Benzin,Stationwagon,1598cc,106hp,ÖndenÇekiş,62lt,Belirtilmemiş,TakasaUygun,Galeriden,"8,8lt"
299,0,21117756,06Ekim2022,Toyota,Corolla,1.6GL,1992,300.000km,Düz,LPG&Benzin,Hatchback/5,1587cc,105hp,ÖndenÇekiş,50lt,"1değişen,12boyalı",TakasaUygun,Galeriden,NaN


In [72]:
display(all_car_df)

,İlanNo,İlanTarihi,Marka,Seri,Model,Yıl,Kilometre,VitesTipi,YakıtTipi,KasaTipi,MotorHacmi,MotorGücü,Çekiş,YakıtDeposu,Boya-değişen,TakasaUygun,Kimden,Ort.YakıtTüketimi
0,21074793,29Eylül2022,Renault,R19,1.6EuropaRNE,1998,247.000km,Düz,LPG&Benzin,Sedan,1598cc,90hp,ÖndenÇekiş,55lt,Belirtilmemiş,TakasaUygunDeğil,Sahibinden,NaN
0,21075160,29Eylül2022,Fiat,Palio,1.2S,2001,203.000km,Düz,LPG&Benzin,Hatchback/5,1242cc,74hp,ÖndenÇekiş,51lt,"3değişen,9boyalı",TakasaUygun,Sahibinden,"7,2lt"
0,21075144,29Eylül2022,Mercedes-Benz,E,200,1987,358.000km,Düz,LPG&Benzin,Sedan,1997cc,109hp,Arkadanİtiş,NaN,Belirtilmemiş,TakasaUygunDeğil,Sahibinden,NaN
0,21075036,29Eylül2022,Tofaş,Doğan,S,1994,85.000km,Düz,LPG&Benzin,Sedan,1581cc,83hp,Arkadanİtiş,50lt,Belirtilmemiş,TakasaUygunDeğil,Sahibinden,"6,5lt"
0,21075209,29Eylül2022,Renault,Megane,2.0RXT,1998,358.125km,Düz,LPG&Benzin,Sedan,1998cc,115hp,ÖndenÇekiş,60lt,Belirtilmemiş,TakasaUygunDeğil,Sahibinden,"8,4lt"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,20566589,17Eylül2022,Tofaş,Doğan,SLXie,2000,99.999km,Düz,LPG&Benzin,Sedan,1581cc,96hp,Arkadanİtiş,45lt,Belirtilmemiş,TakasaUygun,Galeriden,"6,6lt"
0,21005745,19Eylül2022,Opel,Vectra,2.0CD,1996,228.500km,Düz,LPG&Benzin,Sedan,1998cc,115hp,ÖndenÇekiş,NaN,"2değişen,11boyalı",TakasaUygunDeğil,Sahibinden,NaN
0,21106260,04Ekim2022,Daewoo,Nubira,1.6SX,1998,275.000km,Düz,LPG&Benzin,Stationwagon,1598cc,106hp,ÖndenÇekiş,62lt,Belirtilmemiş,TakasaUygun,Galeriden,"8,8lt"
0,21117756,06Ekim2022,Toyota,Corolla,1.6GL,1992,300.000km,Düz,LPG&Benzin,Hatchback/5,1587cc,105hp,ÖndenÇekiş,50lt,"1değişen,12boyalı",TakasaUygun,Galeriden,NaN
